In [130]:
import pandas as pd
import boto3
import io
import numpy as np
import os
from boto3.s3.transfer import TransferConfig

In [131]:
import sagemaker
import boto3
from sagemaker import get_execution_role
import botocore.config
region = "ap-southeast-1"
bucket = "test-data"
session = sagemaker.Session()


In [2]:
get_execution_role()

In [132]:
role = get_execution_role()
config = botocore.config.Config(read_timeout=5000)

# This is the client we will use to interact with SageMaker AutoPilot
sm = boto3.Session().client(service_name='sagemaker',region_name=region, config=config)

In [133]:
bucket = "test-data"
prefix = "sagemaker/annoy-knn-transformer"

In [293]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/annoy-knn-test:latest'.format(account, region)
print(image)

In [294]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.serializers import CSVSerializer
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.deserializers import StringDeserializer
from sagemaker.deserializers import CSVDeserializer

from sagemaker.amazon.amazon_estimator import get_image_uri


def trained_estimator_from_hyperparams(s3_train_data, hyperparams, output_path, s3_test_data=None):
    knn = sagemaker.estimator.Estimator(
        image,
        get_execution_role(),
        instance_count=1,
        instance_type="ml.m5.2xlarge",
        volume_size=100,
        output_path=output_path,
        sagemaker_session=sagemaker.Session(),
    )
    knn.set_hyperparameters(**hyperparams)

    fit_input = {"training": s3_train_data}
    if s3_test_data is not None:
        fit_input["test"] = s3_test_data
    knn.fit(fit_input)
    return knn

In [407]:
import datetime
time = datetime.datetime.now().strftime("%m-%d-%Y-%H-%M-%S")
prefix_key = "annoy-knn-" + str(time)
knn_output_path = "s3://test-data/annoy-knn/nns/" + prefix_key

In [408]:
hyperparams = {
    "feature_dim": 80,
    "nns": 200,
    "no_of_trees":600,
    "metric":"dot",
    "search_k" : -1,
    "knn_all": False, 
    "on_disk" : True,
    "output_path" : knn_output_path,
    "include_distances": True,
    "search_by_vector" : True,
    "retrain":False,
    "model_name":"annoy-test"
}

In [ ]:
s3_train_data = f"s3://test-data/annoy-user-item/data"
model_output_path = f"s3://test-data/annoy-knn/model/output"
knn_estimator = trained_estimator_from_hyperparams(
    s3_train_data, hyperparams, model_output_path, s3_test_data=None
)

In [573]:
str(rows[0])

In [400]:
def predictor_from_estimator(knn_estimator, estimator_name, instance_type, instance_count, endpoint_name=None):
    knn_predictor = knn_estimator.deploy(
        initial_instance_count=instance_count, model_name = estimator_name, instance_type=instance_type, endpoint_name=endpoint_name
    )
    knn_predictor.serializer = CSVSerializer()
    knn_predictor.deserializer = CSVDeserializer()
    return knn_predictor

In [410]:
import datetime
time = datetime.datetime.now().strftime("%m-%d-%Y-%H-%M-%S")
model_name = "annoy-knn-predictor" + str(time)
print(model_name)

In [435]:
import time

instance_type = "ml.c5d.4xlarge"
instance_count = 1
#model_name = "annoy-knn-%"
endpoint_name = "knn-%s" % (str(time.time()).replace(".", "-"))
print("setting up the endpoint..")
predictor = predictor_from_estimator(
    knn_estimator, model_name, instance_type, instance_count, endpoint_name=endpoint_name,
)

In [501]:
def invoke():
    start_time = datetime.datetime.now()
    payload = '{payload}'
    response = runtime.invoke_endpoint(EndpointName='{endpoint_name}' ,
                                           ContentType='text/csv',
                                           Body=payload)
    invoke_time = datetime.datetime.now() - start_time
    return [str(invoke_time), response['ResponseMetadata']['HTTPStatusCode']]

In [502]:
from joblib import Parallel, delayed
import multiprocessing
cpu_count = multiprocessing.cpu_count()
Parallel(n_jobs=cpu_count, require="sharedmem")(
                delayed(invoke)() for i in range(20)
    )

In [438]:
response['Body'].read().decode()